## FAFB14 mirror landmarks
In this notebook, we will generate a faster "shortcut" to a symmetrized FAFB14 space.

In [1]:
import numpy as np
import pandas as pd

import flybrains
import navis

In [2]:
# This is in nm coordinates
bbox = np.array(flybrains.FLYWIRE.boundingbox).reshape(3, 2)
bbox

array([[192200, 853686],
       [ 75853, 398832],
       [  2007, 271205]])

In [3]:
center = bbox[0][0] + (bbox[0][1] - bbox[0][0]) / 2
center

522943.0

In [6]:
# Sample points within the volume
offset = 2500 * 8
res = 2500 * 8
sample = np.mgrid[bbox[0][0]-offset:bbox[0][1]+offset:res,
                  bbox[1][0]-offset:bbox[1][1]+offset:res,
                  max(0, bbox[2][0]-offset):bbox[2][1]+offset:res].reshape(3,-1).T

# Drop points outside the volume
sample = sample[navis.in_volume(sample, flybrains.FLYWIRE.mesh_whole_brain)]
sample.shape

(3390, 3)

In [7]:
sample

array([[172200, 175853, 180000],
       [172200, 195853, 180000],
       [172200, 215853, 160000],
       ...,
       [872200, 355853, 160000],
       [872200, 375853, 120000],
       [872200, 375853, 140000]])

In [8]:
# Split points into left and right
right = sample[sample[:, 0] < center]
left = sample[sample[:, 0] > center]
right.shape, left.shape

((1629, 3), (1761, 3))

In [ ]:
# The sampled coordinates
fig = navis.plot3d([flybrains.FLYWIRE, left], scatter_kws={'color': 'r', 'name': 'left'}, inline=False)
navis.plot3d([right], scatter_kws={'color': 'g', 'name': 'right'}, fig=fig)

In [11]:
# Now mirror left coordinates onto the right
left_mirr = navis.mirror_brain(left.astype(np.float64), template='FLYWIRE', via='JRC2018F', verbose=True)
left_mirr

Transform path: FLYWIRE -> FLYWIREum -> JRC2018F
Transform path: JRC2018F -> FLYWIREum -> FLYWIRE


array([[-520288.476 ,   18430.6625,  197890.976 ],
       [-518511.836 ,   18493.8173,  217759.547 ],
       [-521506.073 ,   38387.3709,  197981.081 ],
       ...,
       [-872027.451 ,  293835.153 ,  289084.155 ],
       [-876798.209 ,  313665.524 ,  249437.22  ],
       [-875021.572 ,  313728.693 ,  269305.736 ]])

In [12]:
# Flip back without compensating
left_flipback = navis.mirror_brain(left_mirr, template='FLYWIRE', warp=False)
left_flipback

array([[1566174.476 ,   18430.6625,  197890.976 ],
       [1564397.836 ,   18493.8173,  217759.547 ],
       [1567392.073 ,   38387.3709,  197981.081 ],
       ...,
       [1917913.451 ,  293835.153 ,  289084.155 ],
       [1922684.209 ,  313665.524 ,  249437.22  ],
       [1920907.572 ,  313728.693 ,  269305.736 ]])

In [ ]:
# The flipped coordinates
fig = navis.plot3d([flybrains.FLYWIRE, left_flipback], scatter_kws={'color': 'r', 'name': 'left'}, plotly_inline=False)
navis.plot3d([right], scatter_kws={'color': 'g', 'name': 'right'}, fig=fig)

In [13]:
# Bring it together
left_source = pd.DataFrame(left, columns=['x', 'y', 'z']).round().astype(int)
left_target = pd.DataFrame(left_flipback, columns=['x_sym', 'y_sym', 'z_sym']).round().astype(int)
left_lm = pd.concat((left_source, left_target), axis=1)

right_source = right_target = pd.DataFrame(right, columns=['x', 'y', 'z']).round().astype(int)
right_lm = pd.concat((right_source, right_target), axis=1)
right_lm.columns = left_lm.columns

lm = pd.concat([left_lm, right_lm], axis=0).reset_index(drop=True)
lm

,x,y,z,x_sym,y_sym,z_sym
0,532200,55853,120000,1566174,18431,197891
1,532200,55853,140000,1564398,18494,217760
2,532200,75853,120000,1567392,38387,197981
3,532200,75853,140000,1565615,38451,217850
4,532200,75853,160000,1563839,38514,237718
...,...,...,...,...,...,...
3385,512200,355853,140000,512200,355853,140000
3386,512200,355853,160000,512200,355853,160000
3387,512200,355853,180000,512200,355853,180000
3388,512200,355853,200000,512200,355853,200000


In [14]:
lm.to_csv('FLYWIRE_symmetrize_landmarks_nm.csv', index=False)

In [15]:
# Generate landmarks for FAFB too
from fafbseg import flywire  # this registers the FAFB<->FLYWIRE transformation on spine
lm_fafb = lm.copy()
lm_fafb[['x', 'y', 'z']] = navis.xform_brain(lm[['x', 'y', 'z']].values, 'FLYWIRE', 'FAFB14')
lm_fafb[['x_sym', 'y_sym', 'z_sym']] = navis.xform_brain(lm[['x_sym', 'y_sym', 'z_sym']].values, 'FLYWIRE', 'FAFB14')
lm_fafb.to_csv('FAFB_symmetrize_landmarks_nm.csv', index=False)

Transform path: FLYWIRE -> FLYWIREraw -> FAFB14raw -> FAFB14
